In [1]:
import platform
import itertools

# py3.5 中无法使用这个，因为不能使用rooback，不支持
# import sqlalchemy.pool as pool


if platform.system() == 'Linux':
    from impala import dbapi
else:
    dbapi = None


class ImpalaSql(object):

    def __init__(self, host='', port=21050, **args):
        self.host = host
        self.port = port
        self._db_operation_timeout = 300
        self.conn = None
        self._connect()
        # self.conn_pool = pool.manage(dbapi, pool_size=20, max_overflow=40,
        #                              timeout=self._db_operation_timeout,
        #                              recycle=1800)

        try:
            self.reconnect()
        except Exception as e:
            print('connect Error : %s ' % e)

    def _connect(self):
        self.conn = dbapi.connect(host=self.host, port=self.port)

    def __del__(self):
        self.close()

    def reconnect(self):
        """Closes the existing database connection and re-opens it."""
        # self.conn = connect(host=self.host,
        #                     port=self.port)
        if getattr(self, "conn", None) is None:
            self._connect()

    def close(self):
        if getattr(self, "conn", None) is not None:
            try:
                self.conn.close()
            except Exception as e:
                print(e)
            self.conn = None

    def _ensure_connected(self):
        if (self.conn is None):
            self.reconnect()

    def _cursor(self):
        self._ensure_connected()
        return self.conn.cursor()

    def query(self, sql):
        """
        get query result for hive
        It's very well if return a generator when dealing with big data.
        """
        query_datas = []
        cursor = self.conn.cursor()
        try:
            cursor.execute(sql)
            column_names = [x[0] for x in cursor.description]
            query_datas = [Row(itertools.izip(column_names, row))
                           for row in cursor]
        except Exception as e:
            raise Exception("""
                errr_info:%s \n
                sql: %s
            """ % (str(e), sql))
        finally:
            cursor.close()

        return query_datas

    def exe_sql(self, sqls):
        """
        update, insert, delete
        """
        sqls = sqls.strip().strip(';')

        cursor = self.conn.cursor()
        try:
            for sql in sqls.split(";"):
                cursor.execute(sql)
        except Exception as e:
            raise Exception("""
                errr_info:%s \n
                sql: %s
            """ % (str(e), sql))
        finally:
            cursor.close()

        return 0


class Row(dict):
    """access object like dict"""
    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)


if __name__ == '__main__':
    print('hello')
    impala = ImpalaSql()
    print(impala)
    sql = """
        SELECT
            act_id id
        FROM
            stage.rt_click_event
        GROUP BY
            act_id
    """
    print(sql)
    for x in impala.query(sql):
        print(x)


hello


AttributeError: 'NoneType' object has no attribute 'connect'